# Example-07: Multipole element factory

In [1]:
# In this example multipole factory is illustrated

In [2]:
import jax
from jax import jit
from jax import jacrev

from elementary.util import ptc
from elementary.util import beta
from elementary.multipole import multipole_factory

jax.numpy.set_printoptions(linewidth=256, precision=12)

In [3]:
# Set data type

jax.config.update("jax_enable_x64", True)

In [4]:
# Set device

device, *_ = jax.devices('cpu')
jax.config.update('jax_default_device', device)

In [5]:
# Set initial condition

(q_x, q_y, q_s) = qs = jax.numpy.array([-0.01, 0.005, 0.001])
(p_x, p_y, p_s) = ps = jax.numpy.array([0.001, 0.001, -0.0001])
x = jax.numpy.hstack([qs, ps])

In [6]:
# Define generic sextupole element

gamma = 10**6
element = jit(multipole_factory(beta=beta(gamma), gamma=gamma, order=2**1, iterations=100, settings=dict(ns=2**1)))

In [7]:
# Compare with PTC

length = jax.numpy.float64(0.25)

kq_n = jax.numpy.float64(-2.0)
kq_s = jax.numpy.float64(+1.5)
ks_n = jax.numpy.float64(-50.0)
ks_s = jax.numpy.float64(+75.0)
ko_n = jax.numpy.float64(-100.0)
ko_s = jax.numpy.float64(+500.0)

print(res := element(x, length, kq_n, kq_s, ks_n, ks_s, ko_n, ko_s))
print(ref := ptc(x, 'quadrupole', {'l': float(length), 'knl': f'{{0.0,{float(kq_n*length)}, {float(ks_n*length)}, {float(ko_n*length)}}}', 'ksl': f'{{0.0,{float(kq_s*length)}, {float(ks_s*length)}, {float(ko_s*length)}}}'}, gamma=gamma))
print(jax.numpy.allclose(res, ref))

[-0.010195569406  0.004634631697  0.000999272345 -0.002587384238 -0.003898816771 -0.0001        ]
[-0.010195569406  0.004634631697  0.000999272345 -0.002587384238 -0.003898816771 -0.0001        ]
True


In [8]:
# Differentiability

print(jacrev(element)(x, length, kq_n, kq_s, ks_n, ks_s, ko_n, ko_s))
print()

print(jacrev(element, 1)(x, length, kq_n, kq_s, ks_n, ks_s, ko_n, ko_s))
print()

[[ 1.058513409508e+00  1.641931362481e-02  0.000000000000e+00  2.548728245136e-01  1.371326922561e-03  1.953576941092e-04]
 [ 1.642278685170e-02  9.426914513911e-01  0.000000000000e+00  1.371531111274e-03  2.452402248601e-01  3.641387859174e-04]
 [ 1.183448542480e-04 -1.074854969909e-04  1.000000000000e+00  2.074101491120e-04  3.548722956512e-04  1.451592817300e-06]
 [ 4.713919361344e-01  1.315473343451e-01  0.000000000000e+00  1.058139548854e+00  1.647371074327e-02  2.099929059292e-05]
 [ 1.316213051530e-01 -4.522013363616e-01  0.000000000000e+00  1.647803989950e-02  9.430571887065e-01 -3.375925241435e-05]
 [-6.192865626461e-23  4.745338216997e-23  0.000000000000e+00 -2.057783281331e-23 -6.577614444340e-24  1.000000000000e+00]]

[-2.587671336604e-03 -3.899249387940e-03 -1.095003441633e-05 -1.481596650448e-02 -1.916336194108e-02  6.001644775824e-24]

